In [141]:
import io
import os
import regex

import eventlet
from eventlet.green.urllib import request

from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline

import numpy as np
import pandas as pd

## Wybór i przygotowanie danych

Do zaprezentowania klasyfikacji tekstów niezbędnym elementem są oczywiście same teksty. Ponieważ z oczywistych względów nie mogłem oprzeć przykładów na tekstach które są  poufne (a do tej kategorii zaliczają się np. zgłoszenia w systemach CRM) do wykorzystania zostały tylko publicznie dostępne treści. Ponieważ zależało mi na tym aby teksty były polskie oraz łatwo dostępne zdecydowałem się na publikacje w serwisie [wolneletury.pl]. Zadanie będzie polegało na ustaleniu nazwiska autora na podstawie jednego zdania. 

W pierwszym kroku należy pobrać treści książek oraz podzielić je na pojedyncze zdania i oczywiście do każdego zdania przypisać autora. Wybór padł na następujące publikacje oraz autorów:
[wolneletury.pl]: http://wolnelektury.pl

In [8]:
book_files={
    "Mickiewicz": [
        "https://wolnelektury.pl/media/book/txt/pan-tadeusz.txt",
        "https://wolnelektury.pl/media/book/txt/dziady-dziady-widowisko-czesc-i.txt",
        "https://wolnelektury.pl/media/book/txt/dziady-dziadow-czesci-iii-ustep-do-przyjaciol-moskali.txt",
        "https://wolnelektury.pl/media/book/txt/ballady-i-romanse-pani-twardowska.txt",
        "https://wolnelektury.pl/media/book/txt/ballady-i-romanse-powrot-taty.txt",
        "https://wolnelektury.pl/media/book/txt/ballady-i-romanse-switez.txt",
        "https://wolnelektury.pl/media/book/txt/dziady-dziady-poema-dziady-czesc-iv.txt",
    ],
    "Sienkiewicz": [
        "https://wolnelektury.pl/media/book/txt/quo-vadis.txt",
        "https://wolnelektury.pl/media/book/txt/sienkiewicz-we-mgle.txt",
        "https://wolnelektury.pl/media/book/txt/potop-tom-pierwszy.txt",
        "https://wolnelektury.pl/media/book/txt/potop-tom-drugi.txt",
        "https://wolnelektury.pl/media/book/txt/potop-tom-trzeci.txt",
    ],
    "Orzeszkowa": [
        "https://wolnelektury.pl/media/book/txt/orzeszkowa-kto-winien.txt",
        "https://wolnelektury.pl/media/book/txt/nad-niemnem-tom-pierwszy.txt",
        "https://wolnelektury.pl/media/book/txt/nad-niemnem-tom-drugi.txt",
        "https://wolnelektury.pl/media/book/txt/nad-niemnem-tom-trzeci.txt",
        "https://wolnelektury.pl/media/book/txt/gloria-victis-dziwna-historia.txt",
        "https://wolnelektury.pl/media/book/txt/z-pozogi.txt",
        "https://wolnelektury.pl/media/book/txt/pani-dudkowa.txt",
        "https://wolnelektury.pl/media/book/txt/dymy.txt",
        "https://wolnelektury.pl/media/book/txt/syn-stolarza.txt",
        "https://wolnelektury.pl/media/book/txt/dobra-pani.txt",
        "https://wolnelektury.pl/media/book/txt/cnotliwi.txt",
        "https://wolnelektury.pl/media/book/txt/kilka-slow-o-kobietach.txt",
        "https://wolnelektury.pl/media/book/txt/patryotyzm-i-kosmopolityzm.txt",
        "https://wolnelektury.pl/media/book/txt/julianka.txt",
    ],
    "Prus": [
        "https://wolnelektury.pl/media/book/txt/lalka-tom-drugi.txt",
        "https://wolnelektury.pl/media/book/txt/lalka-tom-pierwszy.txt",
        "https://wolnelektury.pl/media/book/txt/antek.txt",
        "https://wolnelektury.pl/media/book/txt/katarynka.txt",
        "https://wolnelektury.pl/media/book/txt/prus-anielka.txt",
        "https://wolnelektury.pl/media/book/txt/prus-placowka.txt",
        
    ],
    "Reymont": [
        "https://wolnelektury.pl/media/book/txt/ziemia-obiecana-tom-pierwszy.txt",
        "https://wolnelektury.pl/media/book/txt/chlopi-czesc-pierwsza-jesien.txt",
        "https://wolnelektury.pl/media/book/txt/reymont-chlopi-zima.txt",
        "https://wolnelektury.pl/media/book/txt/chlopi-czesc-trzecia-wiosna.txt",
        "https://wolnelektury.pl/media/book/txt/chlopi-czesc-czwarta-lato.txt",
    ]
}


Wybrane pliki pobierzemy do katalogu data. Do pobrania wykorzystałem bibliotekę [eventlet](http://eventlet.net/doc/) która pozwala na zrównoleglenie intensywnych operacji IO (w tym wypadku pobierania danych z Internetu) z wykorzystaniem tzw. [zielonych wątków](https://pl.wikipedia.org/wiki/Green_thread) (greent hreads). Jest to technika którą w pythonie implementuje się z wykorzystaniem tzw. [współprogramów](https://pl.wikipedia.org/wiki/Wsp%C3%B3%C5%82program) (coroutines).

In [17]:
def fetch(url):
    file_path = os.path.join("data",os.path.basename(url))
    if os.path.exists(file_path):
        return None, None
    data = request.urlopen(url).read()
    return file_path, data

for author in book_files:
    pool = eventlet.GreenPool()
    
    for file_path, data in pool.imap(fetch, book_files[author]):
        if file_path:
            with open(file_path, mode="wb") as f:
                f.write(data)
print ("DONE")

DONE


## Wstępna obróbka i analiza
Tak pobrane pliki z książkami musimy podzielić na zdania, które będziemy wykorzystywać do budowy klasyfikatora. Przy okazji dokonamy ich wstępnej obróbki: zamienimy litery na małe, usuniemy ewentualne znaki specjalne, nadmiarowe spacje itp. Jest to często spotykany jednak dość arbitralny sposób postępowania który w sposób nieodwracalny usuwa z dokumentów (w założeniu mało istotne) informacje. Jako ćwiczenie pozostawię zbadanie wpływu sposobu obróbki na ostateczne rezultat

In [116]:
# output corspus file with one sentence per line
def preprocess_file(file_path=None, file_url=None):
    if not file_path and file_url:
        file_path = os.path.join("data",os.path.basename(file_url))
        
    text = open(file_path,'rb').read().decode("utf-8").lower()

    text = regex.sub(u"[^ \n\p{Latin}\-'.?!]", " ",text)
    text = regex.sub(u"[ \n]+", " ", text) # Squeeze spaces and newlines
    text = regex.sub(r"----- ta lektura.*","", text) # remove footer

    return [regex.sub(r"^ ","",l) for l in regex.split('\.|,|\?|!|:',text)]


def get_book_df(document, author):
    return pd.DataFrame({
        'author': pd.Series(len(document)*[author]),
        'txt': pd.Series(document),
    })
    
book_lines_df = pd.concat([
    get_book_df(preprocess_file(file_url=url),author=author) 
        for author in book_files for url in book_files[author] 
])

book_lines_df.head()

,author,txt
0,Mickiewicz,adam mickiewicz pan tadeusz czyli ostatni zajazd na litwie isbn - - - - księga pierwsza gospodarstwo powrót panicza spotkanie się pierwsze w pokoiku drugie u stołu ważna sędziego nauka o grzeczności podkomorzego uwagi polityczne nad modami początek sporu o kusego i sokoła żale wojskiego ostatni woźny trybunału rzut oka na ówczesny stan polityczny litwy i europy litwo
1,Mickiewicz,ojczyzno moja
2,Mickiewicz,ty jesteś jak zdrowie ile cię trzeba cenić ten tylko się dowie kto cię stracił
3,Mickiewicz,dziś piękność twą w całej ozdobie widzę i opisuję bo tęsknię po tobie
4,Mickiewicz,panno święta co jasnej bronisz częstochowy i w ostrej świecisz bramie


Sprawdźmy ile mamy linii danych dla poszczególnych autorów:

In [202]:
book_lines_df.groupby('author').count()

words                                                   
               count       mean        std  min  25%   50%   75%    max
author                                                                 
Mickiewicz    5093.0  16.103868  13.951437  0.0  6.0  13.0  23.0  145.0
Orzeszkowa   22177.0  19.400821  17.257917  0.0  6.0  15.0  27.0  219.0
Prus         31033.0  12.131570  10.041673  0.0  5.0  10.0  17.0  133.0
Reymont      24107.0  16.359398  18.880965  0.0  5.0  10.0  21.0  316.0
Sienkiewicz  40390.0  13.702377  12.037282  0.0  5.0  10.0  19.0  146.0

Jak widać liczba linii dość mocno się różni. Jest to ważna informacja gdyż [niezrównoważone klasy mają znaczny wpływ na rezultaty osiągane przez znaczną część klasyfikatorów](https://www.svds.com/tbt-learning-imbalanced-classes/). Zanotujmy tą informację aby ją później wykorzystać. Obejrzyjmy też statystyki dotyczące ilości wyrazów w zdaniu.

In [118]:
book_lines_df['words'] = book_lines_df['txt'].apply(
    lambda x: len(x.split())
)
book_lines_df.groupby('author')['words'].describe()

,count,mean,std,min,25%,50%,75%,max
author,,,,,,,,
Mickiewicz,5093.0,16.103868,13.951437,0.0,6.0,13.0,23.0,145.0
Orzeszkowa,22177.0,19.400821,17.257917,0.0,6.0,15.0,27.0,219.0
Prus,31033.0,12.131570,10.041673,0.0,5.0,10.0,17.0,133.0
Reymont,24107.0,16.359398,18.880965,0.0,5.0,10.0,21.0,316.0
Sienkiewicz,40390.0,13.702377,12.037282,0.0,5.0,10.0,19.0,146.0


Pierwsza obserwacja jest taka, że  niektóre linie mają zerową długość. Trzeba je więc usunąć. Kolejną jest ciekawostka że Prus miał bardzo spójny styl, najniższą średnią długość zdania (12.13 wyrazów) i najniższe odchylenie standardowe (nie był tez zwolennikiem długich zdań - najdłuższe miało  "zaledwie" 133 wyrazy podczas gry u Reymonta było prawie 2 i pół raza dłuższe. Podobnie wygląda też kwestia na 98 percentylu:

In [121]:
book_lines_df.groupby('author')['words'].quantile(0.98)

author
Mickiewicz     54.0
Orzeszkowa     67.0
Prus           39.0
Reymont        72.0
Sienkiewicz    47.0
Name: words, dtype: float64

Wystarczy ciekawostek, posprzątajmy dane i weźmy się za przygotowanie modelu.

In [113]:
book_lines_df=book_lines_df[~(book_lines_df['words']==0)]
book_lines_df.groupby('author')['words'].describe()

,count,mean,std,min,25%,50%,75%,max
author,,,,,,,,
Mickiewicz,5079.0,16.148258,13.944973,1.0,6.0,13.0,23.0,145.0
Orzeszkowa,22087.0,19.479875,17.248458,1.0,7.0,15.0,27.0,219.0
Prus,30941.0,12.167642,10.034745,1.0,5.0,10.0,17.0,133.0
Reymont,24079.0,16.378421,18.883692,1.0,5.0,10.0,21.0,316.0
Sienkiewicz,40262.0,13.745939,12.031542,1.0,5.0,10.0,19.0,146.0


## Ekstrakcja cech i budowa modelu

Jednym z najprostszych sposobów na przeprowadzenie klasyfikacji tekstów jest wykorzystanie [regresji logistycznej](https://pl.wikipedia.org/wiki/Regresja_logistyczna). Zanim jednak zabierzemy się do budowy naszego modelu nie zapomnijmy o podzieleniu naszych danych na testowe i treningowe abyśmy mogli zweryfikować wyniki na danych out of sample. Do trenowania wykorzystamy 90% danych, pozostawimy do przeprowadzenia testu na zakończeniu. Przy okazji ważna uwaga, ponieważ chcemy zachować w zbiorach wynikowych proporcje pomiędzy klasami takie jak w danych źródłowych wykorzystamy stratyfikację, czyli najpierw podzielimy zbiór źródłowy na oddzielne zbiory dla każdego autora, na każdym z nich oddzielimy po 10% na zbiór testowy a następnie połączymy odpowiednie części z powrotem. Oczywiście na zakończenie oba zbiory (które formalnie w naszej implementacji są przechowywane jako lista, mają więc ustaloną kolejność) potasujemy. Na szczęście wszystkie te operacje wykona za nas funkcja: [`sklearn.model_selection.train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [154]:
train_df, test_df = train_test_split(
    book_lines_df, 
    test_size=0.1,
    stratify=book_lines_df['author'],
)

Skoro mamy już podzielone dane to pozostaje pytanie, jak wprowadzimy nasz tekst do modelu matematycznego? W Machine Learningu zagadnienie to nosi nazwę [Feature Extraction](https://en.wikipedia.org/wiki/Feature_extraction) i [Feature Engineeringu](https://en.wikipedia.org/wiki/Feature_engineering) (istnieje między nimi szereg subtelnych różnic ale odsyłam do definicji gdyż nie będę ich tu teraz objaśniał). Jest to bardzo obszerny temat, powstało nań wiele książek i publikacji, w naszym prostym przykładzie użyjemy jednego z najprostszych możliwych (a jednocześnie niekoniecznie najgorszych) sposobów zamiany testu na liczby. W pierwszym kroku zbudujemy słownik składający się ze wszystkich słów występujących w tekście. Następnie w każdej próbce (zdaniu) przypiszemy wektor długości takiej jak liczba unikalnych słów i na każdej pozycji odpowiadającej określonemu słowu umieścimy liczbę odpowiadającą ilości wystąpień danego słowa w próbce. Proste, prawda? Taki wektor, jak łatwo sobie wyobrazić, w większości składa się z zer. Całą tę operację wykonuje za nas jedna funkcja:
[`sklearn.feature_extraction.text.CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [184]:
vectorizer = CountVectorizer()
vectorizer.fit(train_df['txt'])
sample_sentence = train_df.iloc[2]['txt']
print ("Extracting vector for sentence: '{}'".format(sample_sentence))
vectorizer.transform([sample_sentence])

Extracting vector for sentence: 'po dziwie'


<1x131588 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

Jak widać przykładowe zdanie, składające się ze 2 wyrazów zostało zapisane jako wektor o długości 131588 elementów. Aby przechować taki obiekt w sposób efektywny w pamięci wykorzystano sparse matrix z 2 elementami. Dzięki temu uniknięto konieczności przechowywania 131586 zer :) Przy okazji kolejna ważna informacja, domyślnie tokenizer jako tokeny traktuje wyrazy o minimum 2 znakach więc wszystkie krótsze słowa (podobnie jak znaki specjalne) zostały zignorowane. W słowniku nie znajdziemy więc "i", "na", "od", "po" itp. Tego typu słowa mają zazwyczaj niewielką wartość jeśli chodzi o klasyfikację (występują z podobnym prawdopodobieństwem we wszystkich rodzajach tekstów).

Jak zapewne zauważyliście do budowy słownika wykorzystałem tylko zbiór treningowy. Dla czego nie cały? Otóż jeśli jakieś słowo występuje wyłącznie w zbiorze testowym nie ma sensu dodawać go do słownika gdyż cecha ta i tak nie stanie się elementem modelu. Ponieważ model nic nie nauczył się na temat tego słowa w danych treningowych nie będzie w stanie tej informacji wykorzystać mimo  iż zostanie ona zakodowana w wektorze wejściowym.

Skoro mamy już sposób kodowania zdań na liczby przekształćmy nasze dane, stwórzmy model regresji logistycznej i przeprowadźmy jego dopasowanie ("trenowanie" nie byłoby tu właściwym słowem gdyż jest to proces deterministyczny).

In [245]:
X_train=vectorizer.transform(train_df['txt'])
X_test=vectorizer.transform(test_df['txt'])
model = LogisticRegression(class_weight='balanced', dual=True)
model.fit(X_train, train_df['author'])

LogisticRegression(C=1.0, class_weight='balanced', dual=True,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

Pierwsze dwie operacje zamieniają nasze testy na zbiory wektorów wejściowych. Słowo wyjaśnienia należy się linii trzeciej w której tworzymy model regresji liniowej. Pierwszy z parametrów pomaga zrównoważyć nierównomierne ilości tekstów poszczególnych autorów przypisując im wagi odwrotnie proporcjonalne do częstotliwości występowania danej klasy. Drugi pozwala na wewnętrzne wykorzystanie innego sposobu implementacji algorytmu regresji logistycznej, który jest znacznie szybszy jeśli liczba cech przewyższa ilość próbek (w naszym przypadku mamy 131588 słów w słowniku, czyli kodowanych cech oraz 110520 zdań zbiorze treningowym).

Mając gotowy, dopasowany model sprawdźmy jego jakość na danych testowych:

In [246]:
model.score(X_test, test_df['author'])

0.76123778501628669

Wynik 76% wydaje się być całkiem niezły biorąc pod uwagę że wykorzystaliśmy jeden z najprostszych sposobów kodowania danych, jeden z najprostszych modeli klasyfikacyjnych a to wszystko na domyślnych ustawieniach. Wyobraźmy sobie że sami stajemy przed takim zadaniem i na podstawie zaledwie kilki słów, do tego w losowej kolejności (model zna tylko ilość wystąpień, nie zna kolejności słów!), musimy określić do którego z 5 autorów należy. Nie wygląda to na proste zadanie. Jako ćwiczenie dla czytelników pozostawię weryfikację wyników dla zdań o określonej minimalnej długości.

Trzeba też pamiętać, że accuracy bardzo często [nie jest dobrą miarą](https://www.svds.com/classifiers2/) oceny jakości modelu. Bez wchodzenia w zbyt wiele detali nadmienię że podobnie jest w naszym przypadku. Jako przykład niech posłużą bardziej szczegółowe wyniki miar [precision](https://en.wikipedia.org/wiki/Precision_and_recall), [recall](https://en.wikipedia.org/wiki/Precision_and_recall) (inaczej [sensitivity](https://en.wikipedia.org/wiki/Sensitivity_and_specificity), [czułość](https://pl.wikipedia.org/wiki/Czu%C5%82o%C5%9B%C4%87_i_swoisto%C5%9B%C4%87)), i [f1](https://en.wikipedia.org/wiki/F1_score) dla poszczególnych klas:

In [247]:
from sklearn import metrics
target = test_df['author']
predicted = model.predict(X_test)
print (metrics.classification_report(target, predicted, digits=4))


             precision    recall  f1-score   support

 Mickiewicz     0.6286    0.5521    0.5879       509
 Orzeszkowa     0.7394    0.7344    0.7369      2218
       Prus     0.7527    0.7112    0.7314      3103
    Reymont     0.7553    0.7578    0.7565      2411
Sienkiewicz     0.7955    0.8428    0.8185      4039

avg / total     0.7598    0.7612    0.7600     12280



Widać wyraźnie że najgorzej rozpoznawane są zdania Mickiewicza, który był najsłabiej reprezentowany. Tylko w 55% były one prawidłowo rozpoznane (czułość), z 63% precyzją. Najlepiej, oczywiście, model radzi sobie z Sienkiewiczem dla którego mieliśmy najwięcej próbek.

W następnej części postaram się opisać kilka sposobów na poprawienie powyższych rezultatów zarówno po stronie cech jak i modelu, pokażę jak możemy połączyć etapy obróbki danych i budowy modelu w jeden proces oraz postaram się napisać odrobinę więcej na temat sposobów ewaluacji modelu.